# 3.allow user to select reference corpus

#Feature3

In [28]:
dataset1 = "Brown corpus"
dataset2 = "Enron corpus"
while 1:
    selected_dataset = input(f'Select Courpus you want to use: \n 1: {dataset1} 2: {dataset2}\n')
    selected_dataset = int(selected_dataset)
    if(selected_dataset == 1 or selected_dataset ==2):
        exec_command = "print(f'SUCCESS: You chose: " + str(selected_dataset) + " " + "{dataset" + str(selected_dataset) + "}')"
        exec(exec_command)
        break
    print('Please input decimal number\n')
    try:
        selected_dataset = int(selected_dataset)
    except:
        print('Please input decimal number\n')



SUCCESS: You chose: 2 Enron corpus


## Remove NaN and Change Enron Documents into one Document

In [29]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
def rmNan(df):
    #remove nan
    for i, msg in enumerate(df['body']):
        # print(i, msg)
        if msg is np.nan:
            df = df.drop(i)
    for i, msg in enumerate(df['body']):
        if msg is np.nan:
            print(i, msg) 

    #merge documents into one
    str_all_document=''
    for index, record in df.iterrows():
        str_all_document = str_all_document + str(record[1])
    del df
    
    return pd.DataFrame({"author":["ENRON DATASET"],
                        "body": [str_all_document]})
            

## Make DataFrame, Q, K1, K2, and Enron or Brown Datasets

In [30]:
import pandas as pd
import numpy as np

def makeDataset(datasetnum = 0):
    if(datasetnum == 2):
        df = pd.read_csv("preprocessed_enron.csv")
        df = rmNan(df)
    elif(datasetnum== 1):
        f = open('Brown_ca_ca01.txt', 'r')
        data = f.read()
        f.close()
        df = pd.DataFrame({"author":["BROWN DATASET"],
                          "body": [data]})
    return df.reset_index(drop=True)

In [31]:
#Make df_Q dataset 
f = open('Q_dataset.txt', 'r')
data = f.read()
f.close()
df_Q = pd.DataFrame({"author":["Q DATASET"],
                    "body": [data]})
                    

In [32]:
#Make df_K1 dataset 
f = open('K1_dataset.txt', 'r')
data = f.read()
f.close()
df_K1 = pd.DataFrame({"author":["K1 DATASET"],
                    "body": [data]})


In [33]:
#Make df_K2 dataset 
f = open('K2_dataset.txt', 'r')
data = f.read()
f.close()
df_K2 = pd.DataFrame({"author":["K2 DATASET"],
                    "body": [data]})

In [34]:
#Make df_ref dataset 
df_ref = makeDataset(selected_dataset)

In [35]:
df = pd.concat([df_Q,df_K1, df_K2, df_ref])
df = df.reset_index(drop=True)
del df_Q, df_K1,df_K2,df_ref

In [36]:
df

,author,body
0,Q DATASET,"\n\nHowever, there are frequent situations whe..."
1,K1 DATASET,Download\n\nSource\n\nPDF\nActions\n Copy Pr...
2,K2 DATASET,\n\nWith the rapid growth of the information c...
3,ENRON DATASET,Please view the summary tab on each worksheet ...


# 1. count, list and order the frequency of words 

#Feature1

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
def tokenizeFunc(documents):
    # documents = df['body'].tolist()
    tf_vectorizer = CountVectorizer()
    tf_vectors = tf_vectorizer.fit_transform(documents)         # word frequency list
    return tf_vectors, tf_vectorizer
# for i, msg in enumerate(df['body']):
#     # print(i, msg)
#     if msg is np.nan:
#         print(i, msg)
#tfidf_vectorizer = TfidfVectorizer()
#tfidf_vectors = tfidf_vectorizer.fit_transform(documents) # keyword frequency list
# tf_vectorizer = CountVectorizer()
# tf_vectors = tf_vectorizer.fit_transform(documents)         # word frequency list
# del tf_vectorizer
# del tfidf_vectorizer# データ分割r

### Make All datasets a list and make tf vector and tf vectrizer

In [52]:
documents=[
    df['body'][0], #df_Q['body'][0],
    df['body'][1], #df_K1['body'][0],
    df['body'][2], #df_K2['body'][0],
    df['body'][3], #df_ref['body'][0],]
]
tf_vectors, tf_vectorizer = tokenizeFunc(documents)


In [53]:
#[remove]Dataframeや単語リストが一つのDFで十分な場合削除　7/25日米田

# tf_vectors_Q, tf_vectorizer_Q = tokenizeFunc(df_Q)
# tf_vectors_K1, tf_vectorizer_K1 = tokenizeFunc(df_K1)
# tf_vectors_K2, tf_vectorizer_K2 = tokenizeFunc(df_K2)
# tf_vectors_ref, tf_vectorizer_ref = tokenizeFunc(df_ref)

# words_Q=tf_vectorizer_Q.get_feature_names_out()
# words_K1=tf_vectorizer_K1.get_feature_names_out()
# words_K2=tf_vectorizer_K2.get_feature_names_out()
# words_K3=tf_vectorizer_ref.get_feature_names_out()

## Make a words dictionary in all documents 

In [54]:
# 作成された辞書を作る　:トレインデータ・テストデータ両方に対応
words=tf_vectorizer.get_feature_names_out()

## Make the Words frequency matrix 

### This Matrix's row indices ared corresponding with a document in the 

In [46]:
#First row: dataset Q
#Second row: dataset K1
#Third row: dataset k2
#Fourth row: dataset ref
df

,author,body,tf
0,Q DATASET,"\n\nHowever, there are frequent situations whe...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,K1 DATASET,Download\n\nSource\n\nPDF\nActions\n Copy Pr...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,K2 DATASET,\n\nWith the rapid growth of the information c...,"[0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
3,ENRON DATASET,Please view the summary tab on each worksheet ...,"[199, 73, 1, 2, 1, 0, 4, 1, 3, 280, 8, 3, 3, 1..."


### This Matrix's col indices are corresponding with a word in the above document

In [62]:
words

10475

## Insert Words Frequemcy Vector into 'tf' Column in DF for Each Dataset

In [55]:
tf_mat = tf_vectors.toarray()
del tf_vectors
df['tf'] = tf_mat.tolist()

In [68]:
df

,author,body,tf
0,Q DATASET,"\n\nHowever, there are frequent situations whe...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,K1 DATASET,Download\n\nSource\n\nPDF\nActions\n Copy Pr...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,K2 DATASET,\n\nWith the rapid growth of the information c...,"[0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
3,ENRON DATASET,Please view the summary tab on each worksheet ...,"[199, 73, 1, 2, 1, 0, 4, 1, 3, 280, 8, 3, 3, 1..."


In [293]:
#[remove]ここの記述で必要な部分は上記に記述。そのた不要であれば削除

#tfidf_mat = tfidf_vectors.toarray() # dead every time
#del tfidf_vectors
# tf_mat = tf_vectors.toarray()
# del tf_vectors
# df['tf'] = tf_mat.tolist()
#df['tfidf'] = tfidf_mat.tolist()

In [294]:
#[remove]今回はからのデータセットがそんざいしないため。確認後削除

# 0 ベクトルを消去 Normalization のため
# for i, vec in enumerate(df['tf']):
#     if sum(vec) == 0:
#         df = df.drop(i)

# df = df.reset_index(drop=True)

# 2. count, list and order the frequency of keywords
#Feature2

# 5. display the first 20 words of each dataset 

#Feature5

In [56]:
def dispAndMakeWordFreq(df, words, author = 0):
    data = df.loc[author]
    freq = data['tf']
    wf = pd.DataFrame({'words': words, 'frequency': freq})
    wordli = []
    freqli =[]
    for key, data in wf.iterrows():
        if(int(data[1]) != 0):
            wordli.append(data[0])
            freqli.append(data[1])
        
    return pd.DataFrame({'words': wordli, 'frequency': freqli})
    


## Words Frequency of Dataset Q

In [63]:
wf_list_Q = dispAndMakeWordFreq(df,words, author = 0)

wf_list_Q = wf_list_Q.sort_values('frequency', ascending=False)
wf_list_Q

851

## Words Frequency of Dataset K1

In [69]:
wf_list_K1 = dispAndMakeWordFreq(df,words, author = 1)

wf_list_K1 = wf_list_K1.sort_values('frequency', ascending=False)
wf_list_K1.head(20)

,words,frequency
590,the,78
418,of,66
51,and,50
299,in,42
605,to,39
69,as,32
391,music,25
237,for,17
324,is,17
82,be,16


## Words Frequency of Dataset K2

In [298]:
wf_list_K2 = dispAndMakeWordFreq(df,words, author = 2)

wf_list_K2 = wf_list_K2.sort_values('frequency', ascending=False)
wf_list_K2.head(20)

,words,frequency
1030,the,383
751,of,161
1056,to,128
165,and,121
581,in,119
619,is,91
555,high,76
433,eo,56
1116,with,56
503,for,55


## Words Frequency of Dataset ref

In [70]:
wf_list_ref = dispAndMakeWordFreq(df,words, author = 3)
wf_list_ref = wf_list_ref.sort_values('frequency', ascending=False)
wf_list_ref.head(20)

,words,frequency
8636,the,7050
8758,to,5262
4014,folder,4149
8477,synchronizing,4024
1374,and,3127
3580,enron,2866
6348,of,2685
3426,ect,2454
4823,in,2132
9495,you,2097


# Display the first 20 keywords of each dataset 

In [341]:
def dispAndMakeSharedKeywordFreq(df,df_ref):
    df = df[df['words'].isin(df_ref['words'])] #filtering with the words in df2
    df_ref = df_ref[df_ref['words'].isin(df['words'])] #filtering with the words in df
    #now the words in df and df2 are same
    #sort words in the alphabetical order to become the same words as the same rows
    df = df.sort_values('words')
    df_ref = df_ref.sort_values('words')
    #merge df2 frequency to df1 
    df['ref_frequency'] = list(df_ref['frequency'])
    df['shared_word_keyword_frequency'] = (df['frequency'] + df['ref_frequency'])
    return df

## Keyword Frequency in Dataset Q and Ref

In [347]:
#KF = Shared Keyword Frequency
KF_QandRef = dispAndMakeSharedKeywordFreq(wf_list_Q,wf_list_ref)
KF_QandRef.sort_values('frequency', ascending=False).head(20)

,words,frequency,ref_frequency,shared_word_keyword_frequency
751,the,186,154,340
517,of,89,65,154
66,and,74,40,114
769,to,70,55,125
315,for,55,22,77
380,in,54,40,94
105,be,45,21,66
527,or,40,4,44
82,are,31,5,36
405,is,31,12,43


## Keyword Frequency in Dataset K1 and Ref

In [348]:
#KF = Shared Keyword Frequency
KF_QandRef = dispAndMakeSharedKeywordFreq(wf_list_K1,wf_list_ref)
KF_QandRef.sort_values('frequency', ascending=False).head(20)

,words,frequency,ref_frequency,shared_word_keyword_frequency
590,the,78,154,232
418,of,66,65,131
51,and,50,40,90
299,in,42,40,82
605,to,39,55,94
69,as,32,13,45
324,is,17,12,29
237,for,17,22,39
82,be,16,21,37
419,on,15,10,25


## Keyword Frequency in Dataset K2 and Ref

In [349]:
#KF = Shared Keyword Frequency
KF_QandRef = dispAndMakeSharedKeywordFreq(wf_list_K2,wf_list_ref)
KF_QandRef.sort_values('frequency', ascending=False).head(20)

,words,frequency,ref_frequency,shared_word_keyword_frequency
1030,the,383,154,537
751,of,161,65,226
1056,to,128,55,183
165,and,121,40,161
581,in,119,40,159
619,is,91,12,103
1116,with,56,6,62
503,for,55,22,77
188,at,50,9,59
177,are,37,5,42


# 7. display the shared words in the first 20 words/keywords of each dataset
#Feature7

## Shared Words Frequency in Dataset Q and K1

In [346]:
#SWF = Shared Word Frequency
SWF_QandK1 = dispAndMakeSharedKeywordFreq(wf_list_Q,wf_list_K1)
SWF_QandK1.sort_values('frequency', ascending=False).head(20)

,words,frequency,ref_frequency,shared_word_keyword_frequency
751,the,186,383,569
517,of,89,161,250
66,and,74,121,195
769,to,70,128,198
315,for,55,55,110
380,in,54,119,173
105,be,45,19,64
527,or,40,10,50
405,is,31,91,122
82,are,31,37,68


## Shared Wowrds Frequency in Dataset Q and K2

In [345]:
#SWF = Shared Word Frequency
SWF_QandK2 = dispAndMakeSharedKeywordFreq(wf_list_Q,wf_list_K2)
SWF_QandK2.sort_values('frequency', ascending=False).head(20)

,words,frequency,ref_frequency,shared_word_keyword_frequency
751,the,186,383,569
517,of,89,161,250
66,and,74,121,195
769,to,70,128,198
315,for,55,55,110
380,in,54,119,173
105,be,45,19,64
527,or,40,10,50
405,is,31,91,122
82,are,31,37,68


## Display the shared keywords in the first 20 keywords of each dataset

## Normalization of Word Frequencies

In [ ]:
# we generaly name 'ntf' for normalized term frequency
normalized_tf_list = []
for row in df['tf']:
    num_words = sum(row)
    normalized_tf = []
    for x in row:
        normalized_tf.append(x/num_words)

    normalized_tf_list.append(normalized_tf)

df['ntf'] = normalized_tf_list

#Tf-idf の代わりに利用する keyness を作る

ここでは　df['keyness'] を作成し追加したい

In [ ]:
import math
# we generaly name 'ntf' for normalized term frequency


# First, create the shared normalized tf vector
shared_ntf = None # shared ntf of all document

matrix = []
for row in df['ntf']:
    matrix.append(row)

np_matrix = np.array(matrix)

mean_vector = np_matrix.mean(axis=0)

shared_ntf = mean_vector.tolist()

def keyness(ntf_vector1, ref_ntf_vector2): # freq_vector1 and freq_vector2 are both already normalized
    keyness_vec = []
    for i, x in enumerate(ntf_vector1):
        if ntf_vector1[i] == 0:
            keyness_vec.append(0)
        else:
            keyness_vec.append(math.log2(ntf_vector1[i]/ref_ntf_vector2[i]))

    return keyness_vec


keyness_mat = []
for ntf_vector in df['ntf']:
    keyness_vec = keyness(ntf_vector, shared_ntf)
    keyness_mat.append(keyness_vec)

# keyness を　Dataframe に追加
df['keyness'] = keyness_mat

In [ ]:
#確認用
i = 22
msg = df['body'][i]
max_value = max(keyness_mat[i])
max_idx = keyness_mat[i].index(max_value)
print(words[max_idx])

print(msg)
#print(df['author'][i])


mechelle
Mechelle,
            I will let you make the call, MW's or $'s. Look at the spreadsheet  below and give me your thoughts.
 
                                                              Chris
              




# Split data into Train Data and Test Data

In [ ]:
from sklearn.model_selection import train_test_split
X_keyness_train, X_keyness_test, Y_keyness_train, Y_keyness_test = train_test_split(df['keyness'],df['author'],test_size=0.2,shuffle=True)
X_tf_train, X_tf_test, Y_tf_train, Y_tf_test = train_test_split(df['tf'],df['author'],test_size=0.2,shuffle=True)

## Check the number of the Authors

In [ ]:
# How many author?
authors = set(Y_keyness_test)
authors_list = [author for author in authors]

# Create Reference_vectors
size: 著者の数

Train データから作る

In [ ]:
# df_X = pd.DataFrame(X_tf_train.values.tolist())
# df_Y = pd.DataFrame(Y_tf_train.values.tolist())


df_train = pd.concat((X_keyness_train, Y_keyness_train.rename('author')), axis=1)
#
reference_vectors = {}
for author in authors:

    df_author = df_train.groupby('author').get_group(author)

    matrix = []
    for row in df_author['keyness']:
        matrix.append(row)

    np_matrix = np.array(matrix)

    mean_vector = np_matrix.mean(axis=0)
    reference_vectors[author] = mean_vector.tolist()

In [ ]:
for author, ref_vec in reference_vectors.items():
    max_value = max(ref_vec)
    max_idx =ref_vec.index(max_value)
    print(f'{author}: {words[max_idx]}')


hunter.shively@enron.com: hunter
j..kean@enron.com: attached
chris.stokley@enron.com: chris
v.weldon@enron.com: charlie
ben.jacoby@enron.com: ben
andy.zipper@enron.com: andy


In [ ]:
authors

{'andy.zipper@enron.com',
 'ben.jacoby@enron.com',
 'chris.stokley@enron.com',
 'hunter.shively@enron.com',
 'j..kean@enron.com',
 'v.weldon@enron.com'}

In [ ]:
# import nltk
# from nltk.corpus import stopwords

# nltk.download('stopwords')
# stop_words = stopwords.words('english')



```
# これはコードとして書式設定されます
```

# Prediction

In [ ]:
# start からend までのwordの配列を返す
def extract_features_words(freq_vector, words, start=0, end=20):

    setX = set(freq_vector) # 最大値を取り出すため set を作成

    count = 0

    result = []

    while count<end:
        max_value = max(setX)
        max_index = freq_vector.index(max_value)
        max_word = words[max_index]

        setX.remove(max_value)

        ### if exclude stopwords
        # if max_word not in stop_words:
        #     if count>= start:
        #         result.append(max_index)
        #     count+=1

        if count>= start:
            result.append(max_word)
        count += 1


    return result

In [ ]:
# start からend までのword IDの配列を返す
def extract_features(freq_vector, words, start=0, end=20):

    setX = set(freq_vector) # 最大値を取り出すため set を作成

    count = 0

    result = []

    while count<end:
        try:
            max_value = max(setX)
        except ValueError:
            return result

        max_index = freq_vector.index(max_value)
        max_word = words[max_index]

        setX.remove(max_value)

        ### if exclude stopwords
        # if max_word not in stop_words:
        #     if count>= start:
        #         result.append(max_index)
        #     count+=1

        if count>= start:
            result.append(max_word)
        count += 1

        #


    return result

In [ ]:
authors

{'andy.zipper@enron.com',
 'ben.jacoby@enron.com',
 'chris.stokley@enron.com',
 'hunter.shively@enron.com',
 'j..kean@enron.com',
 'v.weldon@enron.com'}

In [ ]:
vec = reference_vectors['j..kean@enron.com']
top_words = extract_features_words(vec, words=words,  start=0, end=150)
print(top_words)


['attached', 'we', 'please', 'see', 'ken', 'our', 'my', 'as', 'of', 'california', 'but', 'is', 'have', 'organizations', 'this', 'organization', 'synchronizing', 'management', 'humor', 'campaign', 'for', 'reg', 'rto', 'with', 'folder', 'issues', 'buydown', 'india', 'about', 'associations', 'lists', 'some', 'message', 'objectives', 'security', 'advertising', 'think', 'cftc', 'council', 'and', 'profits', 'contacts', 'press', 'blackberry', 'forms', 'fyi', 'should', 'are', 'brazil', 'federal', 'offline', 'handheld', 'inbox', 'hr', 'wireless', 'demand', 'presentations', 'net', 'budget', 'like', 'steven', 'from', 'was', 'japan', 'be', 'pr', 'corporate', 'crisis', 'committee', 'not', 'wind', 'in', 'rules', 'environmental', 'business', 'he', 'personal', 'make', 'legislation', 'contributions', 'would', 'tax', 'government', 'much', 'speeches', 'tasks', 'ferc', 'communications', 'so', 'excess', 'sec', 'kean', 'rail', 'or', 'responsibility', 'to', 'it', 'his', 'releases', 'what', 'advisory', 'credi

In [ ]:
def get_similarity(feature_vector1,feature_vector2):
    return len(set(feature_vector1) & set(feature_vector2))



In [ ]:

INF = float('inf')

def predict(questioned_vector,reference_vectors):
    start = 0
    end = 20
    suspected = [author for author in authors]
    innocent_list = []
    while(len(suspected) > 1):
        Q_features = extract_features(questioned_vector, words, start, end)
        similarityWithQ = {}

        for author, reference_vector in reference_vectors.items():
            if author in suspected: #
                feature_vector = extract_features(reference_vector, words, start, end)
                score = get_similarity(feature_vector,Q_features)
                similarityWithQ[author]=score

        # innocent_list に含まれない著者の中から1人を選ぶ

        innocent = min(similarityWithQ, key=similarityWithQ.get)
        #print(innocent)
        # print(type(innocent))
        # print(f'{min(similarityWithQ, key=similarityWithQ.get)} may be innocent.')
        suspected.remove(innocent)
        # similarityWithQ[innocent] = INF



        end += 20
    return suspected[0]

In [ ]:
# suspected = [author for author in authors]
# suspected.remove('andy.zipper@enron.com')
# suspected

## Check function in Train Data

In [ ]:
i = 0
bad_guy = predict(df['keyness'][i], reference_vectors)
print(f'bad guy : {bad_guy}')
print('-'*20)
print(f"True author : {df['author'][i]}")

hunter.shively@enron.com
j..kean@enron.com
chris.stokley@enron.com
andy.zipper@enron.com
v.weldon@enron.com
bad guy : ben.jacoby@enron.com
--------------------
True author : chris.stokley@enron.com


# Check function in Test Data

In [ ]:
X_keyness_test

516     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2246    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
271     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1874    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
96      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                              ...                        
1285    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1675    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1217    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
631     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1048    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: keyness, Length: 531, dtype: object

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
all_test_data = len(X_keyness_test)

In [ ]:
match_cnt = 0
all_test_data = len(X_keyness_train)
for i in X_keyness_train.index:
    bad_guy = predict(df['keyness'][i], reference_vectors)
    if df['author'][i] == bad_guy:
        match_cnt = match_cnt + 1

print(f'Math rate is: {match_cnt/all_test_data*100} % ')
    #print(f'bad guy : {bad_guy}')
    #print('-'*20)
    #print(f"True author : {df['author'][i]}")

Math rate is: 51.67216203485634 % 


In [ ]:
# def predict(questioned_vector, reference_vectors):
#     suspected = [author for author in authors]

#     comparedSize = 20
#     while(len(suspected) > 1):




